In [10]:
# Import dependencies
import pandas as pd
import gmaps
import gmaps.datasets
import requests
import matplotlib.pyplot as plt


#import the API key
from config import g_key

%matplotlib inline

In [11]:
# Store the CSV you saved created in part one into a DataFrame
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Kavieng,-2.5744,150.7967,83.84,72,100,10.51,PG,2022-08-05 00:50:29
1,1,Port Augusta,-32.5000,137.7667,59.00,49,72,21.63,AU,2022-08-05 00:50:29
2,2,Kapaa,22.0752,-159.3190,84.52,68,0,10.36,US,2022-08-05 00:46:40
3,3,Bardiyah,31.7561,25.0865,76.75,72,0,9.19,LY,2022-08-05 00:50:30
4,4,Dikson,73.5069,80.5464,42.84,100,100,11.18,RU,2022-08-05 00:45:33


In [12]:
# Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [13]:
# configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [14]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))